# *OpenScope: V2*
<img src="https://static.trademarkia.com/images/73449053" width="380" />

## Run the update/reset shortcut on the desktop before each experiment
***
***
# **Without mouse on stage**

In [2]:
import contextlib
import time

import np_config
import np_jobs
import np_logging
import np_services
import np_session
import np_workflows
from np_workflows import npxc
import np_workflows.experiments.openscope_V2 as V2

from np_services.resources.zro import ZroError 
import contextlib

np_logging.getLogger()

np_workflows.elapsed_time_widget()

***
## Quiet mode
**on**  [*default*]
- error details are hidden
- regular messages displayed (log level = INFO)

**off**
- full error details (with traceback)
- extra messages displayed (log level = DEBUG)

In [3]:
np_workflows.quiet_mode_widget()

ToggleButton(value=True, button_style='info', description='Quiet mode is on', icon='check', tooltip='Quiet mod…

***
## Launch apps via RSC
[optional]

In [3]:
with contextlib.suppress(Exception):
    np_services.start_rsc_apps()

***
## Select mouse and user

In [4]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Check MTrain and select workflow
Re-run cell this cell if mouse ID is changed

In [5]:
np_workflows.mtrain_widget(mouse)

Select type of workflow to run

In [6]:
selected_workflow = V2.V2_workflow_widget(mouse)

***
## Generate new session
Check mouse ID and session are correct: this cell will lock them in!

In [7]:
V2.validate_selected_workflow(selected_workflow.session, mouse)

experiment: V2.Ephys | V2.Hab = V2.new_experiment(mouse, user, selected_workflow.session)
session: np_session.PipelineSession = experiment.session
platform_json: np_session.PlatformJson = experiment.session.platform_json

platform_json.workflow_start_time = npxc.now()
hab: bool = isinstance(experiment, V2.Hab)

12:38 | web | INFO | Ephys(1445658787_366122_20250630) created


***
## Checks before starting

### **Ephys day 2?** 
Don't forget to adjust probe targeting!

In [8]:
np_workflows.check_hardware_widget()

In [9]:
if not hab:
    np_workflows.check_openephys_widget()

***
## Setup, test, reset all components
*This cell must not be skipped!*

In [8]:
with contextlib.suppress(ZroError):
    experiment.initialize_and_test_services()

12:39 | np_services.utils | INFO | mouse_director is running on DT900490
12:39 | np_services.proxies | INFO | Proxy(MouseDirector) initialized: ready for use
12:39 | np_services.utils | INFO | sync_device is running on W10DT26AD0025
12:39 | np_services.proxies | INFO | Proxy(Sync) initialized: ready for use
12:39 | np_services.proxies | INFO | VideoMVR initialized: ready for use
12:39 | np_services.proxies | INFO | ImageMVR initialized: ready for use
12:39 | np_services.proxies | INFO | NewScaleCoordinateRecorder | Test passed
12:39 | np_services.proxies | INFO | NewScaleCoordinateRecorder | Test passed
12:39 | np_services.utils | INFO | camstim_agent is running on W10DT713942
12:39 | np_services.utils | INFO | phidget_server is running on W10DT713942
12:39 | np_services.proxies | INFO | Proxy(SessionCamstim) initialized: ready for use
12:39 | np_services.open_ephys | INFO | OpenEphys | Initializing
12:39 | np_services.open_ephys | INFO | OpenEphys | Testing
12:39 | np_services.open_ep

***
## Dip probes

In [ ]:
if not hab:
    print(np_workflows.dye_info_widget.__doc__)
    np_workflows.dye_info_widget(session)

## Photodoc of probes in dye

In [ ]:
if not hab:
    if platform_json.rig_id == 'NP.1':
        print(str(session) + '_surface-image1-left.png')
    else:
        np_workflows.photodoc_widget('pre_experiment_surface_image')

## Probe depths in dye

In [ ]:
if not hab:
    np_workflows.probe_depth_widget(session)

***
***
# **With mouse on stage**
## Before lowering cartridge

In [ ]:
platform_json.HeadFrameEntryTime = npxc.now()
np_workflows.wheel_height_widget(session)
np_workflows.check_mouse_widget()

***
## When cartridge is lowered

### Set zoom to 4.0 for photodocs of brain
- focus on the brain surface

## Photodoc of brain (tap probes if hab Day1 or Day2)

In [ ]:
platform_json.CartridgeLowerTime = npxc.now()
if platform_json.rig_id == 'NP.1':
    print(str(session) + '_surface-image2-left.png')
else:    
    np_workflows.photodoc_widget('brain_surface_image')

***
## ISI map

In [ ]:
np_workflows.isi_widget(mouse.lims)

***
## Probe insertion

In [ ]:
if not hab:
    platform_json.ProbeInsertionStartTime = npxc.now()

***
## Photodoc before advancing probes

In [ ]:
if not hab:
    if platform_json.rig_id == 'NP.1':
        print(str(session) + '_surface-image3-left.png')
    else:
        np_workflows.photodoc_widget('pre_insertion_surface_image')

***
## Settle timer & insertion notes

- run both cells now: settle timer will start

- fill out probe notes while waiting

- press Save once

- notes are saved when the timer finishes (button will turn green to confirm)

In [ ]:
if not hab:
    np_workflows.insertion_notes_widget(session)

In [ ]:
if not hab:
    np_workflows.print_countdown_timer(minutes=.1 if experiment.workflow.value == 'pretest' else 10)

***
## Photodoc after probes settled, before experiment

In [ ]:
if not hab:
    if platform_json.rig_id == 'NP.1':
        print(str(session) + '_surface-image4-left.png')
    else:
        np_workflows.photodoc_widget('post_insertion_surface_image')

In [ ]:
np_workflows.pre_stim_check_widget()

***
## Start devices recording

In [9]:
last_exception = Exception()
attempts = 3
while attempts:
    np_logging.getLogger().info('Waiting for recorders to finish processing') 
    while not all(r.is_ready_to_start() for r in experiment.recorders):
        time.sleep(1)
    np_logging.getLogger().info('Recorders ready')     
    try:
        experiment.start_recording()
    except AssertionError as exc:
        np_logging.getLogger().info('`experiment.start_recording` failed: trying again')
        attempts -= 1
        last_exception = exc              # exc only exists within the try block
    
    else:
        break
else:
    np_logging.getLogger().error(f'`experiment.start_recording` failed after multiple attempts', exc_info=last_exception)
    raise last_exception

12:42 | root | INFO | Waiting for recorders to finish processing
12:42 | root | INFO | Recorders ready
12:42 | np_services.proxies | INFO | Sync | Starting recording
12:42 | np_services.proxies | INFO | Sync | Verified: file on disk is increasing in size
12:42 | np_services.proxies | INFO | VideoMVR | Starting recording
12:42 | np_services.proxies | INFO | VideoMVR | Verified: 3 cameras recording to disk
12:42 | np_services.open_ephys | INFO | OpenEphys | Starting recording
12:42 | np_services.open_ephys | INFO | OpenEphys | Verified files are increasing in size for all Record Nodes


***
## Start stimulus
mapping | active | injection | active | mapping | opto


Script and params determined by MTrain stage

In [12]:
with contextlib.suppress(ZroError):
    experiment.run_stim()

12:43 | web | INFO | Started session TEST_V2
12:43 | np_services.proxies | INFO | Finalizing SessionCamstim
12:43 | np_services.proxies | INFO | SessionCamstim added new data: []
12:43 | web | INFO | Finished session TEST_V2


## Inject drug or vehicle at 45 minute mark
Record injection time and amount in experiment notes

***
## Stop recording

In [13]:
with contextlib.suppress(ZroError):
    experiment.stop_recording_after_stim_finished()

12:43 | np_services.open_ephys | INFO | OpenEphys | Stopping recording
12:43 | np_services.proxies | INFO | VideoMVR | Stopped recording
12:43 | np_workflows.shared.base_experiments | WARNING | Waiting additional 4 s for MVR to finish writing...
12:43 | np_services.proxies | INFO | Sync | Stopped recording


***
## Before removing probes

In [ ]:
if not hab:
    if platform_json.rig_id == 'NP.1':
        print(str(session) + '_surface-image5-left.png')
    else:
        np_workflows.photodoc_widget('post_stimulus_surface_image')

***
## After fully retracting probes

In [ ]:
if not hab:
    if platform_json.rig_id == 'NP.1':
        print(str(session) + '_surface-image6-left.png')
    else:
        np_workflows.photodoc_widget('post_experiment_surface_image')

***
## After raising cartridge

In [ ]:
platform_json.HeadFrameExitTime = npxc.now()

np_workflows.finishing_checks_widget()

## Finalize

In [14]:
platform_json.workflow_complete_time = npxc.now()

experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

12:44 | np_services.open_ephys | INFO | OpenEphys | Finalizing
12:44 | np_services.open_ephys | INFO | OpenEphys | Starting recording
12:44 | np_services.open_ephys | INFO | OpenEphys | Stopping recording
12:44 | np_services.proxies | INFO | Finalizing SessionCamstim
12:44 | np_services.proxies | INFO | SessionCamstim added new data: []
12:44 | npc_sync.sync | INFO | **********************************************************************
12:44 | npc_sync.sync | INFO | No events on line: vsync_stim
12:44 | npc_sync.sync | INFO | **********************************************************************


AssertionError: Sync file has no events on line vsync_stim

## Copy data

In [15]:
experiment.copy_files()

12:44 | np_workflows.shared.base_experiments | INFO | Sync | Copying files {WindowsPath('//W10DT26AD0025/c$/ProgramData/AIBS_MPE/sync/data/20250630T124223.h5')}
12:44 | np_workflows.shared.base_experiments | INFO | VideoMVR | Copying files {WindowsPath('//DT900490/c$/ProgramData/AIBS_MPE/mvr/data/Face_20250630T124237.json'), WindowsPath('//DT900490/c$/ProgramData/AIBS_MPE/mvr/data/Eye_20250630T124237.mp4'), WindowsPath('//DT900490/c$/ProgramData/AIBS_MPE/mvr/data/Face_20250630T124237.mp4'), WindowsPath('//DT900490/c$/ProgramData/AIBS_MPE/mvr/data/Behavior_20250630T124237.mp4'), WindowsPath('//DT900490/c$/ProgramData/AIBS_MPE/mvr/data/Eye_20250630T124237.json'), WindowsPath('//DT900490/c$/ProgramData/AIBS_MPE/mvr/data/Behavior_20250630T124237.json')}
12:44 | np_workflows.experiments.openscope_V2.main_V2_pilot | WARNING | Did not find stim file on npexp matching the format `YYYYMMDDSSSS_mouseID_foragingID.pkl`
12:44 | np_workflows.shared.base_experiments | INFO | Renamed split ephys fold

In [17]:
session.project

Project('NeuropixelPlatformDevelopment')

# Add to post-experiment pipeline

**hab**
- add session to QC queue

**ephys**
- add session to np-exp upload queue, specifying this rig's Acq as `hostname`
    - ensures checksum-validated copy of ephys data on np-exp
    - then adds session to spike-sorting queue
    - then adds session to QC queue

In [16]:
if hab:
    np_jobs.PipelineQCQueue().add_or_update(session, priority=99)
else:
    np_jobs.PipelineNpexpUploadQueue().add_or_update(session, hostname=np_config.Rig().Acq, priority=99)